In [ ]:
import pandas as pd

# Plotting GBS data 
# https://stackoverflow.com/questions/55776092/plotting-a-graph-by-month-wise-in-pandas
# https://stackoverflow.com/questions/61443949/python-pandas-group-by-then-plot-by-category
import re

gbs_val_descriptions = {
    "01FLOW":"airflow-sensor",
    "01RS-CV":"regelsign-verwarming",
    "01RS-GKW":"regelsign-koeling",
    "01RS-VAV":"regelsign-vav", # Variable air volumewith 
    "01PIR":"pir", # Infrared sensor
    "01RM":"ruimte-modus",
    "01RMT":"co2-ruimte",
    "01SP":"setpoint-co2",
    # OAT
    "TT_01BT": "buiten-temperatuur",
    "MT_01RV": "buiten-rv",
    # LBK
    "VA_01PC": "plant-code",
    # LBK 2 excl
    "PT_01SP": "inblaasdruk-setpoint",
    "PT_01IL": "inblaasdruk",
    "TT_03SPB": "inblaastemperatuur-setpoint-berekend",
    "TT_03IL": "inblaastemperatuur",
    "TT_04RL": "retourlucht-temperatuur",
    "WW_01RS": "warmtewiel-sturing",
    "CV_01RS": "regelafsluiter-verwarmer",
    "CV_02RS": "regelafsluiter-koeler",
}

to_drop = []
# file = "2022w18_GBS_trend_Y03-08.xlsx"
# to_drop = ["Y03VAV0339TE-01BSP.PointValue", 'Y03VAV0339TE-01CPA.PointValue', 'Y03VAV0339TE-01RMT.PointValue', 'Y03VAV0339TE-01SP.PointValue']
file = "2022w18-GBS_trend_OAT .xlsx"
# file = "2022w18-GBS_trend-Y-lbk1.xlsx"

gbs = pd.read_excel(f"../EUR/{file}", parse_dates=[["Date", "Time"]], index_col=0, sheet_name=list(range(4)))
gbs = pd.concat(gbs.values())
print(f"Reading in {file}\nColumns: {list(gbs.columns)}\nDropping: {to_drop}")
gbs.drop(to_drop, axis=1, inplace=True)
def extractFeature(id):
    if "_" in id:
        match = re.findall(r'[0-9]([^\d]*?_.*)\..*', id)[0]
    else:
        match = re.findall(r'-(.*)\..*', id)[0]
    feature = gbs_val_descriptions[match]
    return feature

gbs = gbs.rename(columns=extractFeature)
gbs.index.name = "time"

date_range = str(gbs.index.min()) + ' to ' +str(gbs.index.max())
print(f"Range of dates is: {date_range}")
# What is BSP and CPA?
# gbs.drop(["plant-code"], axis=1, inplace=True)
gbs = gbs.replace({',': '.'}, regex=True)
print(gbs.astype(float).describe())
gbs.head(20)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

fig, ax = plt.subplots()

gbs_co2 = gbs[["co2-ruimte", "setpoint-co2", "pir"]]
gbs_co2.plot(ax=ax)

# Why is pir 0 even during spikes?